In [ ]:
from ultralytics import YOLO
model = YOLO("yolo11x-pose.pt")  # load a pretrained model (recommended for training)


In [ ]:
import pyzed.sl as sl
import cv2
filepath = '/home/emartini/nas/MAEVE/HUMAN_MODEL/ego_view/HD2K_SN14938530_10-49-40.svo2'
input_type = sl.InputType()
input_type.set_from_svo_file(filepath)  #Set init parameter to run from the .svo 
init = sl.InitParameters(input_t=input_type, svo_real_time_mode=False)

init.depth_mode = sl.DEPTH_MODE.NEURAL
cam = sl.Camera()
status = cam.open(init)
print(status,sl.ERROR_CODE.SUCCESS)
# if status != sl.ERROR_CODE.SUCCESS: #Ensure the camera opened succesfully 
#     print("Camera Open", status, "Exit program.")
#     exit(1)


In [ ]:
resolution = cam.get_camera_information().camera_configuration.resolution
low_resolution = sl.Resolution(min(720,resolution.width) * 2, min(404,resolution.height))
svo_image = sl.Mat(min(720,resolution.width) * 2,min(404,resolution.height), sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
runtime = sl.RuntimeParameters()
mat = sl.Mat()

svo_frame_rate = cam.get_init_parameters().camera_fps
nb_frames = cam.get_svo_number_of_frames()
print("[Info] SVO contains " ,nb_frames," frames")
    


In [ ]:
i = 0
while i < nb_frames:  # for 'q' key
    err = cam.grab()
    print(err,sl.ERROR_CODE.SUCCESS)
    if err == sl.ERROR_CODE.SUCCESS:
        cam.retrieve_image(svo_image,sl.VIEW.SIDE_BY_SIDE,sl.MEM.CPU,low_resolution) # retrieve image left and right
        svo_position = cam.get_svo_position()
        cv2.imshow("View", svo_image.get_data()) # display both images to cv2

In [7]:
import pyzed.sl as sl
import cv2, numpy as np
from ultralytics import YOLO
from ultralytics import solutions

model = YOLO("yolo11x-pose.pt")  # load pose estimator

# Initialize instance segmentation object
model_seg = YOLO("yolo11x-seg.pt")
# COLORS = np.random.randint(0, 255, size=(80, 3), dtype=np.uint8)  # 80 COCO classes

yolo_names = ["Nose", "LEye", "REye", "LEar", "REar", "LShoulder", "RShoulder",\
              "LElbow", "RElbow", "Wrist", "Wrist", "LHip", "RHip", "LKnee",\
                  "RKnee", "LAnkle", "RAnkle"]

to_display = [0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1]

def overlay_segmentation_people_only(frame, results):
    if results[0].masks is None:
        return frame

    masks = results[0].masks.data.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy().astype(int)
    names = results[0].names

    overlay = frame.copy()

    for i in range(len(masks)):
        cls_id = classes[i]
        label = names[cls_id]
        if label != "person":
            continue  # skip non-person objects

        color = (0, 255, 0)  # green for people
        mask = masks[i]
        mask_uint8 = (mask * 255).astype(np.uint8)

        contours, _ = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            continue

        # Alpha blend the person mask
        mask_3c = np.stack([mask]*3, axis=-1)
        color_arr = np.array(color, dtype=np.uint8).reshape(1, 1, 3)
        colored_mask = (mask_3c * color_arr).astype(np.uint8)
        colored_mask = cv2.resize(colored_mask, (overlay.shape[1], overlay.shape[0]))

        overlay = cv2.addWeighted(overlay, 1.0, colored_mask, 0.6, 0)

        # Draw label at centroid
        # c = max(contours, key=cv2.contourArea)
        # M = cv2.moments(c)
        # if M["m00"] > 0:
        #     cx = int(M["m10"] / M["m00"])
        #     cy = int(M["m01"] / M["m00"])
        #     cv2.putText(overlay, "person", (cx, cy - 5), cv2.FONT_HERSHEY_SIMPLEX,
        #                 0.6, (255, 255, 255), 2, cv2.LINE_AA)

    return overlay



def draw_skeleton(frame, keypoints, conf_threshold=0.6):
    # For each person in the scene
    for kp in keypoints:
        kps = kp.xy[0].cpu().numpy()  # shape: (17, 2)
        scores = kp.conf[0].cpu().numpy()  # shape: (17,)

        # Draw joints
        for i, (x, y) in enumerate(kps):
            if to_display[i] and scores[i] > conf_threshold:
                cv2.circle(frame, (int(x), int(y)), 20, (0, 255, 0), -1)
                cv2.putText(frame, yolo_names[i]+ " (" + str(round(scores[i],1))+")", (int(x + 5), int(y - 5)),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 1, cv2.LINE_AA)


def main():
    zed = sl.Camera()

    init_params = sl.InitParameters()
    init_params.set_from_svo_file("/home/emartini/nas/MAEVE/HUMAN_MODEL/ego_view/HD2K_SN14938530_10-49-40.svo2")  # Use full absolute path
    init_params.svo_real_time_mode = False

    status = zed.open(init_params)
    if status != sl.ERROR_CODE.SUCCESS:
        print("Failed to open SVO file:", status)
        return

    runtime_params = sl.RuntimeParameters()
    image = sl.Mat()

    # Force decode by grabbing once before setting position
    grab_result = zed.grab(runtime_params)
    if grab_result != sl.ERROR_CODE.SUCCESS:
        print("Initial grab failed:", grab_result)
        return
    total_frames = zed.get_svo_number_of_frames()
    print(f"Total frames: {total_frames}")
    zed.retrieve_image(image, sl.VIEW.SIDE_BY_SIDE)
    height, width = image.get_height(), image.get_width()
    
    # Set up VideoWriter
    output_path = "/home/emartini/nas/MAEVE/HUMAN_MODEL/ego_view/test_egoview_yolo.mp4"
    fps = zed.get_camera_information().camera_configuration.fps
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(output_path, fourcc, fps / 2 , (width, height))

    while True:
        err = zed.grab(runtime_params)
        if err == sl.ERROR_CODE.SUCCESS:
            
            zed.retrieve_image(image, sl.VIEW.SIDE_BY_SIDE)
            frame = image.get_data()
            frame = cv2.cvtColor(frame, cv2.COLOR_RGBA2RGB)

            # Human Pose Estimation
            hpe_results = model(frame)
            keypoints = hpe_results[0].keypoints
            
            results = model_seg(frame)
            frame = overlay_segmentation_people_only(frame, results)
            # Draw skeletons if any detected
            
            if keypoints is not None:
                draw_skeleton(frame, keypoints)
                
            # Write to video file
            writer.write(frame)
            
        elif err == sl.ERROR_CODE.END_OF_SVOFILE_REACHED:
            print("Reached end of file.")
            break
        else:
            print("Grab error:", err)
            break

    zed.close()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


[2025-07-08 13:56:00 UTC][ZED][INFO] Logging level INFO
[2025-07-08 13:56:00 UTC][ZED][INFO] [Init]  Depth mode: NEURAL
[2025-07-08 13:56:00 UTC][ZED][INFO] [Init]  Serial Number: S/N 14938530
Total frames: 631

0: 192x640 6 persons, 5.7ms
Speed: 0.7ms preprocess, 5.7ms inference, 0.7ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 3 persons, 2 dining tables, 1 mouse, 2 cell phones, 1 book, 6.1ms
Speed: 0.6ms preprocess, 6.1ms inference, 0.8ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 6 persons, 6.6ms
Speed: 0.9ms preprocess, 6.6ms inference, 0.7ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 3 persons, 1 dining table, 1 mouse, 2 cell phones, 1 book, 6.0ms
Speed: 0.4ms preprocess, 6.0ms inference, 0.8ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 6 persons, 6.3ms
Speed: 0.6ms preprocess, 6.3ms inference, 0.7ms postprocess per image at shape (1, 3, 192, 640)

0: 192x640 3 persons, 2 dining tables, 1 mouse, 1 cell phone, 1 bo